In [25]:
import numpy as np
from fxpmath import Fxp

In [38]:
class myfi(np.ndarray):
    def __new__(cls, input_array, signed=1, w=32, f=16, like=None, quantize=True, **kwargs): 
        ndarray = np.asarray(input_array).view(cls)
        if isinstance(like, myfi):
            ndarray.config(like.kwargs)
            ndarray.resize(like.signed, like.w, like.f, quantize)            
        else:     
            ndarray.config(kwargs)                       
            ndarray.resize(signed, w, f, quantize)            
        return ndarray

    def resize(self, signed, w, f, quantize=True):
        self.signed = signed
        self.w = w
        self.f = f
        self.i = w-f
        if self.w < self.f:
            raise ValueError("fraction length > word length is not support")
        self.precision = 2**-self.f
        upper = 2**(self.w - self.f - (1 if self.signed else 0))
        self.upper = upper - self.precision 
        self.lower = -2**(self.w-self.f-1) if self.signed else 0    
        
        if quantize:
            print(f'Q: {self.size}')
            b = self.ndarray * (2**self.f)
            if self.rounding == 'nearest':
                store_int = np.round(b)
            elif self.rounding == 'floor':
                store_int = np.floor(b)
            else:
                raise ValueError(f'not support rounding method {self.rounding}')
            # TODO: over/underflow
            if self.overflow == 'warp':
                self[...] = (store_int*self.precision)%upper # this will only reassgin array value, instead of create new myfi instance
            elif self.overflow == 'saturate':
                np.clip((store_int*self.precision), self.lower, self.upper, out=self)
 
        return self

    def config(self, kwargs):
        self.rounding = kwargs.get('rounding', 'nearest')
        self.overflow = kwargs.get('overflow', 'warp')
        self.fixed = kwargs.get('fixed', False)

    @property
    def ndarray(self):
        return self.view(np.ndarray)
    @property
    def int(self):
        return (self.ndarray*2**self.f).astype(int) # return normal ndarray
    @property
    def kwargs(self):
        return {
            'rounding' : self.rounding,
            'overflow' : self.overflow,
            'fixed': self.fixed
        }

    def base_repr(self, base=2, frac_point=False): # return ndarray with same shape and dtype = '<Uw' where w=self.w
        if base == 2:
            add_point = lambda s: s[:-self.f] + '.' + s[-self.f:] if frac_point else s
            func = lambda i: add_point(np.binary_repr(i,width=self.w))    
        else:
            func = lambda i: np.base_repr(i,base=base)
        return np.vectorize(func)(self.int)
    @property
    def bin(self): 
        return self.base_repr(2)
    @property 
    def bin_(self):
        return self.base_repr(2,frac_point=True)
    @property
    def hex(self):
        return self.base_repr(16)

    # overload operands    
    def __repr__(self):
        signed = 's' if self.signed else 'u'
        return f'myfi-{signed}{self.w}/{self.f}: \n' + self.ndarray.__repr__() 
    __str__ = __repr__
    def __getitem__(self, key):
        # let single element index of array return myfi with shape (1,) instead of base dtype element
        return myfi(super().__getitem__(key), like=self)
    def __neg__(self):
        return myfi(super().__neg__(), like=self)
    def __pos__(self):
        return self    
    def __arithmeticADD__(self, func, y):
        y = y if isinstance(y, myfi) else myfi(y, like=self)
        i = max(self.i, y.i)
        f = max(self.f, y.f)
        return myfi(func(y.ndarray), self.signed|y.signed, i+f+1, f, None, False, **self.kwargs)         
    def __arithmeticMUL__(self, func, y):
        y = y if isinstance(y, myfi) else myfi(y, like=self)
        return myfi(func(y.ndarray), self.signed|y.signed, self.w+y.w, self.f+y.f, None, False, **self.kwargs) 
    # arithmetic
    __add__         = lambda self,y: self.__arithmeticADD__(super().__add__, y)
    __radd__        = lambda self,y: self.__arithmeticADD__(super().__radd__, y)
    __iadd__        = lambda self,y: self.__arithmeticADD__(super().__iadd__, y)
    __sub__         = lambda self,y: self.__arithmeticADD__(super().__sub__, y)
    __rsub__        = lambda self,y: self.__arithmeticADD__(super().__rsub__, y)
    __isub__        = lambda self,y: self.__arithmeticADD__(super().__isub__, y)
    __mul__         = lambda self,y: self.__arithmeticMUL__(super().__mul__, y)
    __rmul__        = lambda self,y: self.__arithmeticMUL__(super().__rmul__, y)
    __imul__        = lambda self,y: self.__arithmeticMUL__(super().__imul__, y)
    __truediv__     = lambda self,y: self.__arithmeticMUL__(super().__truediv__, y)
    __rtruediv__    = lambda self,y: self.__arithmeticMUL__(super().__rtruediv__, y)
    __itruediv__    = lambda self,y: self.__arithmeticMUL__(super().__itruediv__, y)
    __floordiv__    = lambda self,y: self.__arithmeticMUL__(super().__floordiv__, y)
    __rfloordiv__   = lambda self,y: self.__arithmeticMUL__(super().__rfloordiv__, y)
    __ifloordiv__   = lambda self,y: self.__arithmeticMUL__(super().__ifloordiv__, y)
    # comparison
    __eq__ = lambda self,y: self.ndarray == y
    __ne__ = lambda self,y: self.ndarray != y
    __ge__ = lambda self,y: self.ndarray >= y
    __gt__ = lambda self,y: self.ndarray >  y
    __le__ = lambda self,y: self.ndarray <= y
    __lt__ = lambda self,y: self.ndarray <  y
    # logical TODO
    # bit operation TODO

    #TODO: casting


x = np.random.rand(3,2,2)
y = myfi(x,1,14,3) # call __new__ and __array_finalize__
print(y.w,y.f,y.precision)
z = y[1:]
print(z.w,z.f,z.precision)
q = myfi(x,like=y)
print(q.w,q.f,q.precision)
w = y[1,1,1]
print(w.w,w.f,w.precision)
# y[1,1,1] # call __array_finalize__
# x.view(myfi) # call __array_fianlize__

Q: 12
14 3 0.125
Q: 8
14 3 0.125
Q: 12
14 3 0.125
Q: 1
14 3 0.125


In [34]:
s = 1
w = 15
f = 8
x = np.linspace(0,100,77)
print('init fx')
fx = myfi(x,s,w,f)
y = 1/3
print('init fy')
fy = myfi(y,s,w,f+2)
xy = x + y
print('fx+fy')
fxy = fx + fy
# xyf = myfi(fx.view(np.ndarray)+y, s,w+1,f)
# print(fxy == xyf)
# print(fxy.ndarray)

init fx
Q: 77
init fy
Q: 1
fx+fy


In [44]:
x = myfi(np.linspace(0,20,100),1,8,4,overflow='saturate')
x

Q: 100


myfi-s8/4: 
array([0.    , 0.1875, 0.375 , 0.625 , 0.8125, 1.    , 1.1875, 1.4375,
       1.625 , 1.8125, 2.    , 2.25  , 2.4375, 2.625 , 2.8125, 3.    ,
       3.25  , 3.4375, 3.625 , 3.8125, 4.0625, 4.25  , 4.4375, 4.625 ,
       4.875 , 5.0625, 5.25  , 5.4375, 5.6875, 5.875 , 6.0625, 6.25  ,
       6.4375, 6.6875, 6.875 , 7.0625, 7.25  , 7.5   , 7.6875, 7.875 ,
       7.9375, 7.9375, 7.9375, 7.9375, 7.9375, 7.9375, 7.9375, 7.9375,
       7.9375, 7.9375, 7.9375, 7.9375, 7.9375, 7.9375, 7.9375, 7.9375,
       7.9375, 7.9375, 7.9375, 7.9375, 7.9375, 7.9375, 7.9375, 7.9375,
       7.9375, 7.9375, 7.9375, 7.9375, 7.9375, 7.9375, 7.9375, 7.9375,
       7.9375, 7.9375, 7.9375, 7.9375, 7.9375, 7.9375, 7.9375, 7.9375,
       7.9375, 7.9375, 7.9375, 7.9375, 7.9375, 7.9375, 7.9375, 7.9375,
       7.9375, 7.9375, 7.9375, 7.9375, 7.9375, 7.9375, 7.9375, 7.9375,
       7.9375, 7.9375, 7.9375, 7.9375])

In [37]:
def wrap(x, signed, n_word): 
    m = (1 << n_word)
    if signed: 
        x = np.array(x).astype(int) & (m - 1) 
        x = np.where(x < (1 << (n_word-1)), x, x | (-m)) 
    else: 
        x = np.array(x).astype(int) & (m - 1) 
    return x

In [58]:
x = np.linspace(0,20,100000)
up=10
down=1
def foo():    
    a = x.copy()
    a[x>up] = up
    a[x<down] = down
def bar():
    x.clip(up,down)
%timeit foo()
%timeit bar()



163 µs ± 2.24 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
734 µs ± 2 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [56]:
%lprun -f foo foo()


Timer unit: 2.43802e-07 s

Total time: 0.000344492 s
File: <ipython-input-55-dd93ecad2cbf>
Function: foo at line 4

Line #      Hits         Time  Per Hit   % Time  Line Contents
     4                                           def foo():    
     5         1        659.0    659.0     46.6      x[x>up] = up
     6         1        754.0    754.0     53.4      x[x<down] = down

In [57]:
%lprun -f bar bar()

Timer unit: 2.43802e-07 s

Total time: 0.000890365 s
File: <ipython-input-55-dd93ecad2cbf>
Function: bar at line 7

Line #      Hits         Time  Per Hit   % Time  Line Contents
     7                                           def bar():
     8         1       3652.0   3652.0    100.0      x.clip(up,down,out=x)